<a href="https://colab.research.google.com/github/Razib99/Amazon_Sales_Sentiment_Analysis_CSE445_ML/blob/main/Amazon_Sales_Sentiment_Analysis_CSE445.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('amazon.csv')

In [ ]:
df.head(5)

NameError: name 'df' is not defined

In [ ]:
df.info()
df.isnull().sum()
df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

In [ ]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 1463 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1463 non-null   object
 1   product_name         1463 non-null   object
 2   category             1463 non-null   object
 3   discounted_price     1463 non-null   object
 4   actual_price         1463 non-null   object
 5   discount_percentage  1463 non-null   object
 6   rating               1463 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1463 non-null   object
 9   user_id              1463 non-null   object
 10  user_name            1463 non-null   object
 11  review_id            1463 non-null   object
 12  review_title         1463 non-null   object
 13  review_content       1463 non-null   object
 14  img_link             1463 non-null   object
 15  product_link         1463 non-null   object
dtypes: object(1

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stop_words]
    return ' '.join(text)

df['cleaned_reviews'] = df['review_title'].apply(preprocess_text)

In [ ]:
def assign_sentiment(rating):
    # Convert 'rating' column to float type
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

    # Round 'rating' column to nearest integer
    df['rating'] = df['rating'].round().astype(int)

    if rating >= 4:
        return 1  # Positive
    elif rating == 3:
        return 0  # Neutral
    else:
        return -1  # Negative

df['sentiment'] = df['rating'].apply(assign_sentiment)

In [ ]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Assuming X_train, X_test, y_train, y_test are already defined

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the models (excluding Linear Regression)
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(n_neighbors=3),  # Adjust n_neighbors to a suitable value
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier()
}

results = {}

# Fit the models and store the results
for name, model in models.items():
    model.fit(X_train, y_train_encoded)
    y_pred = model.predict(X_test)
    y_pred_encoded = label_encoder.inverse_transform(y_pred)
    accuracy = accuracy_score(y_test, y_pred_encoded)
    report = classification_report(y_test, y_pred_encoded, zero_division=0)
    confusion = confusion_matrix(y_test, y_pred_encoded)
    results[name] = {
        'accuracy': accuracy,
        'classification_report': report,
        'confusion_matrix': confusion
    }

# Print the results
for name, result in results.items():
    print(f"Model: {name}")
    print(f"Accuracy: {result['accuracy']}")
    print(f"Classification Report:\n{result['classification_report']}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")

Model: Logistic Regression
Accuracy: 0.9590443686006825
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.96      1.00      0.98       281

    accuracy                           0.96       293
   macro avg       0.48      0.50      0.49       293
weighted avg       0.92      0.96      0.94       293

Confusion Matrix:
[[  0  12]
 [  0 281]]

Model: Naive Bayes
Accuracy: 0.9590443686006825
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.96      1.00      0.98       281

    accuracy                           0.96       293
   macro avg       0.48      0.50      0.49       293
weighted avg       0.92      0.96      0.94       293

Confusion Matrix:
[[  0  12]
 [  0 281]]

Model: SVM
Accuracy: 0.962457337883959
Classification Report:
              precision    recall  f1-score   supp